In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import re

import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim

from dataset import Dataset
from datetime import datetime
from model import Autoencoder
from torch.utils.data import DataLoader

In [2]:
def train(model, train_loader, criterion, optimizer, epochs=20):
    
    print('starting model training...')
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"training on {'cuda' if torch.cuda.is_available() else 'cpu'}")
    model.to(device)
    history = []
    
    for epoch in range(epochs):
        start = time.time()
        model.train()
        training_loss, running_loss = 0.0, 0.0
        
        # Iterando sobre o dataset
        for batch_i, data in enumerate(train_loader):
            X = data['X'].to(device)
            
            # Zero Grad
            optimizer.zero_grad()

            # Forward Pass
            output = model(X)

            # Loss Function
            loss = criterion(output, X)

            # Backward Pass
            loss.backward()
            running_loss += loss.item()
            training_loss += loss.item()

            # Update
            optimizer.step()

            if batch_i % 100 == 99:
                print('Batch: {}, Avg. Loss: {}'.format(batch_i + 1, running_loss / 100))
                running_loss = 0.0
        
        # Epoch results
        training_loss /= len(train_loader)
        print(f'[{round(time.time() - start, 3)} secs] Epoch: {epoch+1}/{epochs}', end='')
        print(f', Training loss: {training_loss}', end='\n\n')
        history.append(training_loss)
    
    print('training finished.')
    
    date = str(datetime.now()).split('.')[0]    
    model_path = f'.\\models\\model_{re.sub(r"[^0-9]", "", date)}.pt'
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': training_loss
    }, model_path)
    
    print('model saved.')
    
    return history

def load_model(path):
    
    model = Autoencoder(input_dim=dim)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    train_loss = checkpoint['loss']
    
    return model, optimizer, epoch, train_loss

In [3]:
train_dataset = Dataset(path='.\\data\\training.h5', key='normal')
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)

dim = train_dataset[0]['X'].shape[0]

model = Autoencoder(input_dim=dim)
print(model)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

Autoencoder(
  (encoder1): Linear(in_features=52, out_features=256, bias=True)
  (encoder2): Linear(in_features=256, out_features=128, bias=True)
  (encoder3): Linear(in_features=128, out_features=26, bias=True)
  (decoder1): Linear(in_features=26, out_features=128, bias=True)
  (decoder2): Linear(in_features=128, out_features=256, bias=True)
  (decoder3): Linear(in_features=256, out_features=52, bias=True)
)


In [ ]:
# Training model
train_loss = train(model, train_loader, criterion, optimizer, epochs=50)

starting model training...
training on cuda
Batch: 100, Avg. Loss: 0.6757068499922753
Batch: 200, Avg. Loss: 0.39336280435323717
Batch: 300, Avg. Loss: 0.311646970808506
Batch: 400, Avg. Loss: 0.2943175122141838
Batch: 500, Avg. Loss: 0.2919827476143837
Batch: 600, Avg. Loss: 0.28560491412878036
Batch: 700, Avg. Loss: 0.27827429831027983
Batch: 800, Avg. Loss: 0.2761135169863701
Batch: 900, Avg. Loss: 0.2751822239160538
Batch: 1000, Avg. Loss: 0.2722132921218872
Batch: 1100, Avg. Loss: 0.2693168011307716
Batch: 1200, Avg. Loss: 0.269123744815588
Batch: 1300, Avg. Loss: 0.26979887127876284
Batch: 1400, Avg. Loss: 0.27036403834819794
Batch: 1500, Avg. Loss: 0.2696216785907745
Batch: 1600, Avg. Loss: 0.26904914289712906
Batch: 1700, Avg. Loss: 0.26893986463546754
Batch: 1800, Avg. Loss: 0.2681207522749901
Batch: 1900, Avg. Loss: 0.2668739515542984
[30.443 secs] Epoch: 1/50, Training loss: 0.30296513886632304

Batch: 100, Avg. Loss: 0.2670369490981102
Batch: 200, Avg. Loss: 0.2674601525068

Batch: 700, Avg. Loss: 0.23444226861000061
Batch: 800, Avg. Loss: 0.23341860830783845
Batch: 900, Avg. Loss: 0.23287246912717818
Batch: 1000, Avg. Loss: 0.23295313939452172
Batch: 1100, Avg. Loss: 0.2333099827170372
Batch: 1200, Avg. Loss: 0.23226645663380624
Batch: 1300, Avg. Loss: 0.23202922984957694
Batch: 1400, Avg. Loss: 0.23117508217692376
Batch: 1500, Avg. Loss: 0.23244280010461807
Batch: 1600, Avg. Loss: 0.23210255041718483
Batch: 1700, Avg. Loss: 0.23082134768366813
Batch: 1800, Avg. Loss: 0.23259472340345383
Batch: 1900, Avg. Loss: 0.23201425164937972
[29.801 secs] Epoch: 10/50, Training loss: 0.23284793870109863

Batch: 100, Avg. Loss: 0.2318622513115406
Batch: 200, Avg. Loss: 0.23106044456362723
Batch: 300, Avg. Loss: 0.23092888534069062
Batch: 400, Avg. Loss: 0.23104309797286987
Batch: 500, Avg. Loss: 0.22986504539847374
Batch: 600, Avg. Loss: 0.22964326038956642
Batch: 700, Avg. Loss: 0.23021557435393333
Batch: 800, Avg. Loss: 0.2304046232998371
Batch: 900, Avg. Loss: 0.2

Batch: 1300, Avg. Loss: 0.2130633381009102
Batch: 1400, Avg. Loss: 0.21273164063692093
Batch: 1500, Avg. Loss: 0.21214801251888274
Batch: 1600, Avg. Loss: 0.212166695445776
Batch: 1700, Avg. Loss: 0.21266652688384055
Batch: 1800, Avg. Loss: 0.21164346486330032
Batch: 1900, Avg. Loss: 0.2115333589911461
[29.953 secs] Epoch: 19/50, Training loss: 0.21252588744722345

Batch: 100, Avg. Loss: 0.21065342396497727
Batch: 200, Avg. Loss: 0.21108262076973916
Batch: 300, Avg. Loss: 0.21012845784425735
Batch: 400, Avg. Loss: 0.21041535809636117
Batch: 500, Avg. Loss: 0.210799620449543
Batch: 600, Avg. Loss: 0.21129563733935355
Batch: 700, Avg. Loss: 0.20992181092500686
Batch: 800, Avg. Loss: 0.21068478599190713
Batch: 900, Avg. Loss: 0.20959248825907706
Batch: 1000, Avg. Loss: 0.21068648084998132
Batch: 1100, Avg. Loss: 0.20951804041862487
Batch: 1200, Avg. Loss: 0.2100693602859974
Batch: 1300, Avg. Loss: 0.2087372548878193
Batch: 1400, Avg. Loss: 0.2089216688275337
Batch: 1500, Avg. Loss: 0.2089

Batch: 1900, Avg. Loss: 0.2016477380692959
[30.31 secs] Epoch: 28/50, Training loss: 0.20226466801387383

Batch: 100, Avg. Loss: 0.2020454251766205
Batch: 200, Avg. Loss: 0.20373900175094606
Batch: 300, Avg. Loss: 0.201545400172472
Batch: 400, Avg. Loss: 0.20266517832875253
Batch: 500, Avg. Loss: 0.20265408098697663
Batch: 600, Avg. Loss: 0.20048748433589936
Batch: 700, Avg. Loss: 0.20089795023202897
Batch: 800, Avg. Loss: 0.20212845981121064
Batch: 900, Avg. Loss: 0.20167502924799918
Batch: 1000, Avg. Loss: 0.20214170694351197
Batch: 1100, Avg. Loss: 0.20244064778089524
Batch: 1200, Avg. Loss: 0.2022572386264801
Batch: 1300, Avg. Loss: 0.20207226216793062
Batch: 1400, Avg. Loss: 0.2021945609152317
Batch: 1500, Avg. Loss: 0.20234355136752127
Batch: 1600, Avg. Loss: 0.20172506406903268
Batch: 1700, Avg. Loss: 0.20276488184928895
Batch: 1800, Avg. Loss: 0.20279771000146865
Batch: 1900, Avg. Loss: 0.2029288338124752
[29.767 secs] Epoch: 29/50, Training loss: 0.2021741970321173

Batch: 100

Batch: 400, Avg. Loss: 0.1998651258647442
Batch: 500, Avg. Loss: 0.20005365952849388
Batch: 600, Avg. Loss: 0.20033441483974457
Batch: 700, Avg. Loss: 0.19899181351065637
Batch: 800, Avg. Loss: 0.19999795243144036
Batch: 900, Avg. Loss: 0.19987766683101654
Batch: 1000, Avg. Loss: 0.19968396589159965
Batch: 1100, Avg. Loss: 0.20030816808342933
Batch: 1200, Avg. Loss: 0.19958114832639695
Batch: 1300, Avg. Loss: 0.19895399257540702
Batch: 1400, Avg. Loss: 0.20014411702752113
Batch: 1500, Avg. Loss: 0.2004253239929676
Batch: 1600, Avg. Loss: 0.19847284406423568
Batch: 1700, Avg. Loss: 0.20038951843976974
Batch: 1800, Avg. Loss: 0.19945936903357506
Batch: 1900, Avg. Loss: 0.20033991619944572
[32.245 secs] Epoch: 38/50, Training loss: 0.19975382532684197

Batch: 100, Avg. Loss: 0.1998999872803688
Batch: 200, Avg. Loss: 0.19812084510922431
Batch: 300, Avg. Loss: 0.19993143305182456
Batch: 400, Avg. Loss: 0.1998563089966774
Batch: 500, Avg. Loss: 0.20033294692635537
Batch: 600, Avg. Loss: 0.19

In [ ]:
fig = go.Figure(data=go.Scatter(y=train_loss, mode='lines+markers'))
fig.show()

In [ ]:
# Loading model
model_path = '.\\models\\model_20210603175854.pt'

In [ ]:
train_dataset = Dataset(path='.\\data\\training.h5', key='normal', simulation=2)
mse = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for data in train_dataset:
    X = data['X'].to(device)
    output = model(X)
    mse.append(criterion(output, X).item())

In [ ]:
fig = go.Figure(data=go.Scatter(y=mse, mode='lines+markers'))
fig.show()

In [ ]:
train_faulty_dataset = Dataset(path='.\\data\\training.h5', key='faulty', simulation=(1, 5))

mse_faulty = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for data in train_faulty_dataset:
    X = data['X'].to(device)
    output = model(X)
    mse_faulty.append(criterion(output, X).item())

In [ ]:
fig = go.Figure(data=go.Scatter(y=mse_faulty, mode='lines+markers'))
fig.show()